In [1]:
# --- Phase 4: Integrate Generative AI (DeepSeek) ---
# This script integrates a Large Language Model (DeepSeek) to provide explanations 
# for the match predictions generated by our Machine Learning model.

import os  # Standard library for operating system interactions
from openai import OpenAI  # Import the OpenAI client library (compatible with DeepSeek)

# ---------------------------------------------------------
# 1. Setup DeepSeek Client
# ---------------------------------------------------------
# We initialize the client that will handle communication with the AI server.
# Note: We use the 'openai' library, but we point 'base_url' to DeepSeek's server.
client = OpenAI(
    # This is your personal API key. It acts as a password to access the service.
    # ideally, keys are stored in environment variables, but hardcoding is fine for this project.
    api_key="sk-e3131dba5a1744be8723a5b0531acd4c",  
    base_url="https://api.deepseek.com"
)

# ---------------------------------------------------------
# 2. Define the Function with Two Templates
# ---------------------------------------------------------
# This function encapsulates the logic for generating text.
# It inputs the match details and the predicted winner, and outputs an explanation.
# The 'template_type' argument allows us to switch between different "personas".
def get_football_advice(home_team, away_team, predicted_winner, template_type="simple"):
    """
    Generates an explanation for a match prediction using DeepSeek.
    Requirements: Must support at least two different prompt templates.
    """
    
    # --- Template 1: Simple / Direct ---
    # Goal: To provide a quick, one-sentence summary suitable for a casual fan.
    if template_type == "simple":
        # System Prompt: Tells the AI *who* it is. Here, just a helpful assistant.
        system_prompt = "You are a helpful football assistant."
        # User Prompt: The specific task. We explicitly ask for "one short sentence".
        user_prompt = f"The match is {home_team} vs {away_team}. My model predicts: {predicted_winner}. Give me one short sentence explaining a key reason why they might win."
    
    # --- Template 2: Expert Analyst ---
    # Goal: To provide a detailed breakdown for a serious analyst or report.
    elif template_type == "expert":
        # System Prompt: We set a specific persona ("senior sports analyst") to influence the tone.
        system_prompt = "You are a senior sports analyst for the Saudi Professional League."
        # User Prompt: We give detailed instructions on structure (analyze strengths, tactics, tone).
        user_prompt = f"""
        Match: {home_team} (Home) vs {away_team} (Away).
        Prediction: {predicted_winner}.
        
        Task: Write a professional match analysis.
        1. Analyze the strengths of the predicted winner.
        2. Discuss any tactical advantages they might have over the opponent.
        3. Keep the tone exciting and professional.
        """
    
    # -----------------------------------------------------
    # Send Request to DeepSeek API
    # -----------------------------------------------------
    # We use a try-except block to handle potential errors (e.g., no internet, wrong key).
    try:
        # client.chat.completions.create is the function that sends the message to the AI.
        response = client.chat.completions.create(
            model="deepseek-chat",  # Specifies we want to use the standard DeepSeek model
            messages=[
                # The 'messages' list sets the context for the AI.
                {"role": "system", "content": system_prompt}, # Sets behavior/persona
                {"role": "user", "content": user_prompt},     # The actual request
            ],
            max_tokens=500,  # Limits output length to save cost (500 tokens is ~400 words)
            temperature=0.7, # Controls creativity. 0.7 is a good balance of creative but focused.
            stream=False     # We want the full answer at once, not word-by-word.
        )
        # Extract and return just the text content of the AI's reply
        return response.choices[0].message.content
        
    except Exception as e:
        # If an error occurs, return the error message instead of crashing.
        return f"Error connecting to DeepSeek: {e}"

# ---------------------------------------------------------
# 3. Test and Compare Templates (Required for Phase 4)
# ---------------------------------------------------------
# This section proves that both templates work and produce different results.

# Example Data (Simulating a prediction output from your Random Forest model)
h_team = "Al-Nassr"
a_team = "Al-Hilal"
pred = "Home Win"

print(f"--- Generating Advice for: {h_team} vs {a_team} (Prediction: {pred}) ---\n")

# --- Test Template 1 (Simple) ---
print(f">>> Template 1 Output (Simple):\n")
# We call our function requesting the 'simple' template
explanation_1 = get_football_advice(h_team, a_team, pred, template_type="simple")
print(explanation_1)

print("\n" + "="*50 + "\n") # Print a separator line for readability

# --- Test Template 2 (Expert) ---
print(f">>> Template 2 Output (Expert Analyst):\n")
# We call our function requesting the 'expert' template
explanation_2 = get_football_advice(h_team, a_team, pred, template_type="expert")
print(explanation_2)

--- Generating Advice for: Al-Nassr vs Al-Hilal (Prediction: Home Win) ---

>>> Template 1 Output (Simple):

Al-Nassr's Cristiano Ronaldo can deliver a decisive moment of individual brilliance against any defense.


>>> Template 2 Output (Expert Analyst):

Of course. Here is a professional match analysis for the upcoming Riyadh Derby.

---

### **Match Analysis: Al-Nassr vs. Al-Hilal - The Kings' Clash**

The stage is set for a seismic encounter in the Saudi Professional League as the iconic Al-Nassr welcomes their eternal rivals, Al-Hilal, to a pulsating Al-Awwal Park. In a clash of titans that promises fireworks, the analysis points towards a decisive **home victory for Al-Nassr**. The energy, the tactics, and the sheer force of individual brilliance are all aligning to create a perfect storm in favor of the Yellow and Blue.

Here is a breakdown of why Al-Nassr is poised to claim the bragging rights.

#### **1. The Unstoppable Force: Deconstructing Al-Nassr's Strengths**

Al-Nassr's 

### 4. Template Comparison & Analysis

**Objective:**
To determine the best way to present match predictions to users, we tested two different prompt templates using the DeepSeek LLM.

**Template 1: Simple / Direct**
* **Goal:** Provide a quick, bite-sized reason for the prediction.
* **Result:** The model successfully generated a single, impactful sentence focusing on a key player (Cristiano Ronaldo).
* **Pros:** Fast to read, good for mobile notifications or quick summaries.
* **Cons:** Lacks depth and context; doesn't explain the "why" in detail.

**Template 2: Expert Analyst**
* **Goal:** Provide a deep, professional match preview.
* **Result:** The model generated a high-quality article titled "The Kings' Clash," with structured sections on "Deconstructing Strengths" and "The Tactical Battleground." It adopted an exciting, professional tone suitable for a sports news site.
* **Pros:** Highly engaging, provides valuable context, and makes the application feel like a premium analysis tool.
* **Cons:** Requires more reading time.

**Conclusion & Selection:**
For our Advice System, we select **Template 2 (Expert Analyst)** as the final choice. [cite_start]Our goal is to provide "detailed advice or explanations"[cite: 87], and the Expert template fulfills this much better than the simple one. It transforms a simple data point ("Home Win") into a comprehensive narrative that offers real value to the user.